# Систематизация фонокардиограмм

Код для систематизации фонокардиограмм с сайта https://archive.physionet.org/pn3/challenge/2016/ .

Фонокардиограммы (ФКГ) разбросаны по папкам. Цель: собрать всех в одну папку, а в названиях файлов указать, есть ли отклонения ритма от нормы, а также для чего будет использоваться данная ФКГ (для обучения или для проверки качества прогноза) 

In [1]:
import os
import pandas as pd
import shutil

Перед запуском следующего кода необходимо в текущей папке (в которой лежит ipynb файл) иметь папку с названием Data, в которой лежит две подпапки training и validation, загруженные по ссылкам https://archive.physionet.org/pn3/challenge/2016/training.zip и https://archive.physionet.org/pn3/challenge/2016/validation.zip соответственно.

In [2]:
path = os.getcwd() + "/Data"

wav_filelist = []
csv_filelist = []

for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.wav'):
            wav_filelist.append(os.path.join(root,file))
        if file.endswith('.csv'):
            csv_filelist.append(os.path.join(root,file))

# for name in wav_filelist:
#     print(name)
# for name in csv_filelist:
#     print(name)

In [3]:
# Количество фонокардиограмм
len(wav_filelist)

3541

Далее генерируется DataFrame с названием всех записей (фонокардиограмм) и соответствующим обозначением: -1 (ритм без отклонений от нормы) или  1 (ритм с отклонением от нормы)

In [4]:
all_frames = []

for path in csv_filelist:
    all_frames.append(pd.read_csv(path, header=None))

all_recordings = pd.concat(all_frames, ignore_index=True)

all_recordings

,0,1
0,b0001,-1
1,b0002,-1
2,b0003,-1
3,b0004,-1
4,b0005,-1
...,...,...
3536,e00622,1
3537,e00627,1
3538,e00648,1
3539,e00657,1


Далее создаётся папка PCG_recordings

In [5]:
path_for_recordings = os.getcwd()+'/PCG_recordings'
if not os.path.exists(path_for_recordings):
    os.makedirs(path_for_recordings)
    print('Папка для записей ФКГ создана')
else:
    print('Папка для записей ФКГ уже существует')

Папка для записей ФКГ создана


Копии всех фонокардиограмм помещаются в созданную папку PCG_recordings. Имя файлов задаётся в следующем формате: (1 или 0 - с отклонениями или без)_(имя файла из PhysioNet)_(test или train - запись будет использоваться для проверки или обучения).

Пример: название файла 0_a0029_test.wav обозначает фонокардиограмму ритма без отклонений от нормы с названием a0029 в базе данных PhysioNet, и данный ритм будет использоваться только при проверке качества прогноза.

Код исполняется около 2,5 минут (копии ФКГ помещаются в соответствующую папку). В дальнешем этот код может быть оптимизирован

In [6]:
for i in range(len(all_recordings)):
    
    if all_recordings[1][i] == 1:
        new_file_name = '1_'+all_recordings[0][i]
    else:
        new_file_name = '0_'+all_recordings[0][i]
    
    file_paths = [s for s in wav_filelist if all_recordings[0][i] in s]
    
    if len(file_paths) == 2:
        old_file_path = [path for path in file_paths if 'valid' in path][0]
        new_file_name = new_file_name + '_test.wav'
    else:
        old_file_path = file_paths[0]
        new_file_name = new_file_name + '_train.wav'
    
    #print(new_file_name)
    
    shutil.copy2(old_file_path, os.getcwd() + '/PCG_recordings/' + new_file_name)